<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/topics/Finetune_XLM_RoBERTa_Topic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Fine-tune and quantize XLM-RoBERTa for topic classification on Russian social media posts

# ========== CONFIGURATION ==========

# Define input files
file_expert_labeled = "512_posts_24_topics.csv"  # expert-labeled dataset
unlabeled_posts = "speechvepkar.csv"

# see 25 XLM-RoBERTa models: Russian + text classification: https://huggingface.co/models?pipeline_tag=text-classification&language=ru&sort=trending&search=XLM-RoBERTa

model_name = "xlm-roberta-base" #👌 (125M+ параметров)
# num_train_epochs=17 при 18 точность перестала расти: 17 и 18 эпоха: точность 0.623656, 19 эпоха - 0.634
# OK: max_length=256
# OK: per_device_train_batch_size=8
# OK: per_device_eval_batch_size =8

# model_name = "mrm8488/XLM-RoBERTa-tiny" # failed to load Урезанная версия XLM-R с многоязычной поддержкой 🔽 22М
# model_name = "mrm8488/xlm-roberta-base-finetuned-HC3-mix" # модель уже содержит классификационную "голову" (head) с 2 выходами, потому что она обучалась на двоичную классификацию (например, truthful / deceptive)
# model_name = "DeepPavlov/xlm-roberta-large-en-ru-mnli" # (head) с 3 выходами предобученной модели, размер [3, 1024] (3 класса), а не 23

# model_name = "cointegrated/rubert-tiny" #👌Крайне низкие метрики (Accuracy 21.5%, F1 ~4–9%) — модель почти не обучается.
# model_name = "distilbert-base-multilingual-cased" #👌метрики неприемлемо низкие (Accuracy ~25%, F1 < 15%).
                                                    # Упрощённый DistilBERT с мультиязычностью	🔽 134М
# model_name = "papluca/xlm-roberta-base-language-detection" # failed, since language-detection

# Уменьшить количество эпох для ускорения 3, 2, 1
num_train_epochs=17 # 21 # 3 10

# Уменьшить max_length при токенизации (todo: подсчитать длину постов)
max_length=512
# max_length=256
# max_length=128

learning_rate=2e-5 # (0.00002) Меньший learning rate означает более медленное обновление весов модели. Это может привести к более стабильному, но медленному обучению.
# learning_rate=3e-5 # (0.00003) Чуть больший learning rate ускоряет обновление весов, что может помочь модели быстрее сходиться, но увеличивает риск "перескакивания" оптимального решения.

per_device_train_batch_size=16
per_device_eval_batch_size =16
#per_device_train_batch_size=8
#per_device_eval_batch_size =8

from config import hugging_face_username

In [2]:
#if model_name == "papluca/xlm-roberta-base-language-detection":
    # Merge expert-assigned topics
    # (1) объединить в одну тему "поэзия Калевала литература" три темы: "поэзия", "Эпос_Калевала", "литература";
    # (2) объединить в одну тему "этнокультура фольклор" две темы: "этнокультура" и "фольклор". Таким образом, исходных тем станет вместо 23 будет 20.
#    merge_map = {
#        'поэзия': 'поэзия Калевала литература',
#        'Эпос_Калевала': 'поэзия Калевала литература',
#        'литература': 'поэзия Калевала литература',
#        'этнокультура': 'этнокультура фольклор',
#        'фольклор': 'этнокультура фольклор'
#    }
#    df['topic'] = df['topic'].replace(merge_map)

In [ ]:
!pip install -U transformers datasets scikit-learn

# ========== IMPORTS ==========
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score
from io import StringIO
import requests

# ========== LOAD AND PREPARE DATA ==========

# Load data labeled by expert from GitHub
response = requests.get(f'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/topics/{file_expert_labeled}')
df = pd.read_csv(StringIO(response.text), delimiter=',', encoding='utf-8')
df = df[df['topic'].str.lower() != 'пусто'].copy()


# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['topic'])

# Train-validation split
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}")

# Optional: you can also encode 'domain' and 'type_group' if you later want to use them
# domain_encoder = LabelEncoder()
# df['domain_id'] = domain_encoder.fit_transform(df['domain'])

# Keep only necessary columns to save memory
train_df = train_df[['text', 'label']].copy()
val_df = val_df[['text', 'label']].copy()

# Convert to HuggingFace Datasets
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
datasets = DatasetDict({"train": train_ds, "validation": val_ds})


# ========== TOKENIZATION ==========
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=max_length)

tokenized_datasets = datasets.map(preprocess_function, batched=True)

# ========== LOAD MODEL ==========
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# ========== METRICS ==========
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
        "f1_weighted": f1_score(labels, preds, average="weighted")
    }

# ========== TRAINING ==========
training_args = TrainingArguments(
    output_dir="./results_xlm_topic",
    eval_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()

# Free memory after training
del tokenized_datasets
torch.cuda.empty_cache()

# Save model and tokenizer
model.save_pretrained("./topic_model_finetuned")
tokenizer.save_pretrained("./topic_model_finetuned")

In [ ]:
# ========== UPLOAD TO HUGGING FACE HUB ==========
from huggingface_hub import HfApi, ModelCard
from datetime import datetime
import os, shutil

repo_name = f"{hugging_face_username}/xlm-roberta-base-topic-classification-2025"
print(f"\nUploading model to Hugging Face Hub: {repo_name}")

os.environ["HF_TOKEN"] = "your token"

# Save model and tokenizer
save_path = "./model_for_upload"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

api = HfApi()
try:
    api.create_repo(
        repo_id=repo_name,
        repo_type="model",
        token=True,
        exist_ok=True
    )

    api.upload_folder(
        folder_path=save_path,
        repo_id=repo_name,
        repo_type="model",
        commit_message="Upload XLM-RoBERTa for topic classification",
        token=True
    )

    card_content = f"""
---
language:
- ru
- multilingual
tags:
- text-classification
- topic-modeling
- xlm-roberta
---

# XLM-RoBERTa Base for Topic Classification

Model fine-tuned for Russian social media topic classification.
- Trained on: {file_expert_labeled}
- Epochs: {num_train_epochs}
- Learning rate: {learning_rate}
"""
    with open(f"{save_path}/README.md", "w") as f:
        f.write(card_content)

    api.upload_file(
        path_or_fileobj=f"{save_path}/README.md",
        path_in_repo="README.md",
        repo_id=repo_name,
        token=True
    )

    print("✅ Upload complete!")
    print(f"Model available at: https://huggingface.co/{repo_name}")

except Exception as e:
    print(f"❌ Error during upload: {e}")
    raise
finally:
    shutil.rmtree(save_path, ignore_errors=True)
    print("Temporary files cleaned up")

In [5]:
# Run inference on new unlabeled data
from transformers import TextClassificationPipeline

# Load unlabeled posts
test_response = requests.get(f'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/{unlabeled_posts}')
test_df = pd.read_csv(StringIO(test_response.text), delimiter=',', encoding='utf-8')
pipe = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    top_k=None,
    device=0 if torch.cuda.is_available() else -1,
    truncation=True,              # 👈 обрезать длинные тексты
    max_length=max_length,               # 👈 безопасный максимум для BERT
    padding=True                  # 👈 чтобы batch работал корректно
)

# Predict topic with score
results = []
for _, row in test_df.iterrows():
    text = row['text']
    if not isinstance(text, str) or not text.strip():
      continue  # Skip invalid text entries

    preds = pipe(text)[0]  # list of dicts [{label: 'LABEL_0', score: ...}, ...]
    best = max(preds, key=lambda x: x['score'])
    topic_label = label_encoder.inverse_transform([int(best['label'].split('_')[-1])])[0]
    results.append({
        "id": row.get('id', None),  # if 'id' exists

        "topic": topic_label, # two calculated fields
        "relatedness": round(best['score'], 4),

        "text": text,
        "date": row.get('date'),
        "likes": row.get('likes'),
        "reposts": row.get('reposts'),
        "views": row.get('views'),
    })

# Save labeled results
pd.DataFrame(results).to_csv("labeled_predictions.csv", index=False, encoding="utf-8")

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
